In [2]:
import requests
import grab
import json
import logging
import codecs
from lxml.etree import ElementTree, fromstring
from bs4 import BeautifulSoup
from tqdm import tqdm
from multiprocessing.dummy import Pool as ThreadPool
from IPython.display import HTML
import re
import os
import time 
import sys
import numpy as np

Получение рейтинга фильма:

In [3]:
url = 'https://www.kinopoisk.ru/film/zvyozdnye-voyny-epizod-6-vozvrashhenie-dzhedaya-1983-447/'
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko)' + 
                  'Chrome/62.0.3202.89 Safari/537.36',
}


g = grab.Grab(headers=headers)
g.go(url)


score = g.xpath_list("//*[@id=\"block_rating\"]/div[1]/div[1]/a/span[1]/text()")
print (score)

C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:12: GrabDeprecationWarning: Call to deprecated function xpath_list. Use grab.doc.select() instead.


['8.157']


Возьмем элементы URL первых 20 фильмов из топ-250 и возьмем актеров игравших в них:

In [132]:
url = 'https://www.kinopoisk.ru/top/'
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_5) AppleWebKit/537.36 (KHTML, like Gecko)' + 
                  'Chrome/61.0.3163.100 Safari/537.36',
}
g = grab.Grab(headers=headers)

MOVIE_URLS = []
actors = []
g.go(url)


for i in range(1,250):
    try:
        g.go(url)
        a = g.xpath_list('//*[@id="top250_place_' + str(i) +'"]/td[2]/a/@href')
        MOVIE_URLS.append(a[0])
        print (a[0])
        g.go('https://www.kinopoisk.ru' + str(a[0]))
        m = []
        for j in range(1,11):
            b = g.xpath_list('//*[@id="actorList"]/ul/li[' + str(j) + ']/a/text()')
            #print(b[0])
            m.append(b[0])
        actors.append(m)
    except IndexError:
        continue

C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:16: GrabDeprecationWarning: Call to deprecated function xpath_list. Use grab.doc.select() instead.


/film/pobeg-iz-shoushenka-1994-326/


C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:22: GrabDeprecationWarning: Call to deprecated function xpath_list. Use grab.doc.select() instead.


/film/zelenaya-milya-1999-435/
/film/forrest-gamp-1994-448/
/film/spisok-shindlera-1993-329/
/film/1-plus-1-2011-535341/
/film/nachalo-2010-447301/
/film/korol-lev-1994-2360/
/film/leon-1994-389/
/film/boycovskiy-klub-1999-361/
/film/ivan-vasilevich-menyaet-professiyu-1973-42664/
/film/zhizn-prekrasna-1997-381/
/film/dostuchatsya-do-nebes-1997-32898/
/film/krestnyy-otec-1972-325/
/film/igry-razuma-2001-530/
/film/prestizh-2006-195334/
/film/kriminalnoe-chtivo-1994-342/
/film/operatsiya-y-i-drugie-priklyucheniya-shurika-1965-42782/
/film/interstellar-2014-258687/
/film/vlastelin-kolec-vozvrashhenie-korolya-2003-3498/
/film/gladiator-2000-474/
/film/nazad-v-budushhee-1985-476/
/film/karty-dengi-dva-stvola-1998-522/
/film/pianist-2002-355/
/film/poymay-menya-esli-smozhesh-2002-324/
/film/v-boy-idut-odni-stariki-1973-25108/
/film/vlastelin-kolec-bratstvo-kolca-2001-328/
/film/brilliantovaya-ruka-1968-46225/
/film/otstupniki-2006-81314/
/film/matrica-1999-301/
/film/vlastelin-kolec-dve-krep

Итого получили подобные данные

In [145]:
print(MOVIE_URLS[:5])

['/film/pobeg-iz-shoushenka-1994-326/', '/film/zelenaya-milya-1999-435/', '/film/forrest-gamp-1994-448/', '/film/spisok-shindlera-1993-329/', '/film/1-plus-1-2011-535341/']


In [146]:
print(actors[:5])

[['Тим Роббинс', 'Морган Фриман', 'Боб Гантон', 'Уильям Сэдлер', 'Клэнси Браун', 'Джил Беллоуз', 'Марк Ролстон', 'Джеймс Уитмор', 'Джеффри ДеМанн', 'Ларри Бранденбург'], ['Том Хэнкс', 'Дэвид Морс', 'Майкл Кларк Дункан', 'Бонни Хант', 'Джеймс Кромуэлл', 'Майкл Джитер', 'Грэм Грин', 'Даг Хатчисон', 'Сэм Рокуэлл', 'Барри Пеппер'], ['Том Хэнкс', 'Робин Райт', 'Салли Филд', 'Гэри Синиз', 'Майкелти Уильямсон', 'Хэйли Джоэл Осмент', 'Майкл Коннер Хэмпфри', 'Ханна Р. Холл', 'Сэм Андерсон', 'Шиван Фэллон'], ['Лиам Нисон', 'Бен Кингсли', 'Рэйф Файнс', 'Кэролайн Гудолл', 'Эмбет Дэвидц', 'Йонатан Сэгаль', 'Малгоша Гебель', 'Шмуэль Леви', 'Марк Иванир', 'Беатриче Макола'], ['Франсуа Клюзе', 'Омар Си', 'Анн Ле Ни', 'Одри Флеро', 'Жозефин де Мо', 'Клотильд Молле', 'Альба Гайя Крагеде Беллуджи', 'Сирил Менди', 'Салимата Камате', 'Абса Дьяту Тур']]


Найдем топ-5 актеров по числу сыгранных ролей в топ 250 фильмах и выпишем эти роли.

In [138]:
actor_movie = []
for i in range(len(actors)):
    for j in range(len(actors[i])):
        actor = []
        actor.append(actors[i][j])
        actor.append(MOVIE_URLS[i])
        actor_movie.append(actor)

In [139]:
import pandas as pd

In [140]:
actor_movie_np = np.array(actor_movie)

In [141]:
df = pd.DataFrame(actor_movie, columns= ['Actor', 'Movie'])
df.head(20)

,Actor,Movie
0,Тим Роббинс,/film/pobeg-iz-shoushenka-1994-326/
1,Морган Фриман,/film/pobeg-iz-shoushenka-1994-326/
2,Боб Гантон,/film/pobeg-iz-shoushenka-1994-326/
3,Уильям Сэдлер,/film/pobeg-iz-shoushenka-1994-326/
4,Клэнси Браун,/film/pobeg-iz-shoushenka-1994-326/
5,Джил Беллоуз,/film/pobeg-iz-shoushenka-1994-326/
6,Марк Ролстон,/film/pobeg-iz-shoushenka-1994-326/
7,Джеймс Уитмор,/film/pobeg-iz-shoushenka-1994-326/
8,Джеффри ДеМанн,/film/pobeg-iz-shoushenka-1994-326/
9,Ларри Бранденбург,/film/pobeg-iz-shoushenka-1994-326/


In [142]:
pd.DataFrame(df.groupby('Actor')['Movie'].count()).sort_values(by='Movie', 
        ascending=False).head(5) 

,Movie
Actor,
Орландо Блум,8
Роберт Де Ниро,7
Брэд Питт,7
Морган Фриман,7
Юрий Никулин,6


Посмотрим в каких фильмах снимался, например, Орландо Блум

In [143]:
df[df['Actor'] == 'Орландо Блум']

,Actor,Movie
184,Орландо Блум,/film/vlastelin-kolec-vozvrashhenie-korolya-20...
254,Орландо Блум,/film/vlastelin-kolec-bratstvo-kolca-2001-328/
292,Орландо Блум,/film/vlastelin-kolec-dve-kreposti-2002-312/
352,Орландо Блум,/film/piraty-karibskogo-morya-proklyatie-chern...
1461,Орландо Блум,/film/belyy-plen-2005-79204/
1884,Орландо Блум,/film/ischeznuvshaya-2014-692861/
2382,Орландо Блум,/film/krupnaya-ryba-2003-9617/
2392,Орландо Блум,/film/maksim-perepelitsa-1955-46512/


In [144]:
df[df['Actor'] == 'Роберт Де Ниро']

,Actor,Movie
380,Роберт Де Ниро,/film/probuzhdenie-1990-2950/
531,Роберт Де Ниро,/film/krestnyy-otec-2-1974-327/
820,Роберт Де Ниро,/film/odnazhdy-v-amerike-1983-469/
1170,Роберт Де Ниро,/film/sherlok-kholms-i-doktor-vatson-korol-sha...
1520,Роберт Де Ниро,/film/korol-govorit-2010-485311/
1730,Роберт Де Ниро,/film/pesn-morya-2014-714248/
1931,Роберт Де Ниро,/film/znakhar-1981-80749/


В текстовом файле находятся элементы url фильмов, для этих фильмов нужно получить средний рейтинг для режиссеров, актеров и рейтинг фильма. Затем составим датасет. Средний рейтинг будем считать переходя по фильмам режиссера и актеров, считая среднее арифметическое.

In [26]:
import time

In [ ]:
url = 'https://www.kinopoisk.ru'
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko)' + 
                  'Chrome/62.0.3202.89 Safari/537.36',
}


proxy = ["89.236.17.106:3128","89.236.17.106:3128","45.32.158.192:3128","51.255.42.237:54566","51.255.42.237:54566","165.227.198.49:80","202.175.61.162:8080","119.15.171.81:65309","202.175.61.162:8080"]

def next_proxy(cur):
    return proxy[(cur + 1) % 9];

f = open("urls.txt", "r")
line = "1"

films_data = []

for i in range(414):
    film_data = []
    line = f.readline()
    film_data.append(line)
    g.go(url + line)

    director = g.xpath_list("//*[contains(@itemprop, 'director')]/a/@href")

    director_score = []
    for i in director:
        g.go(url + i)
        # average_score
        score_list = []
        for i in range(1,10):
            a = g.xpath_list('//*[@id="block_left_padtop"]/div/table/tr/td/table/tr[10]/td/div[2]/div[' + str(i) + ']/div[1]/a')#[0]#.text_content()
            if (len(a)==0):
                continue
            a = a[0].text_content()
            score_list.append(float(a))
        if (len(score_list) == 0):
            continue
        director_score.append(np.mean(score_list))
    
    if (len(director_score) == 0):
        continue
    
    film_data.append(np.mean(director_score))

    g.go(url + line)


    actors = g.xpath_list("//*[@id=\"actorList\"]/ul[1]/li/a/@href")
    actors = actors[:-1]
        
    actor_score = []
    common_count = 0
    for i in actors:
        
        g.go(url + i)
        films_count = g.xpath_list('//*[@id="infoTable"]/table/tr[7]/td[2]/text()[1]')
        if (len(films_count) != 0):
            text = films_count[0].replace('\n','')
            text = text.replace(' ','')
            text = text.replace(',','')
            counts = int(text)
            common_count = common_count + counts


        # average_score
        score_list = []
        for i in range(1,10):
            a = g.xpath_list('//*[@id="block_left_padtop"]/div/table/tr/td/table/tr[9]/td/div[2]/div[' + str(i) + ']/div[1]/a')#[0]#.text_content()
            if (len(a)==0):
                continue
            a = a[0].text_content()
            score_list.append(float(a))
        if (len(score_list) == 0):
            continue
        actor_score.append(np.mean(score_list))
    
    if (len(actor_score) == 0):
        continue
        
    film_data.append(np.mean(actor_score))
        
    g.go(url + line)
    
    score = g.xpath_list("//*[@id=\"block_rating\"]/div[1]/div[1]/a/span[1]/text()")
    if (not len(score)):
        continue
    film_data.append(float(score[0]))
    
    
    films_data.append(film_data)
    time.sleep(1.5)
print(films_data) 
    
f.close()

Преобразуем эти даные в датафрейм

In [30]:
import pandas as pd

In [31]:
X = pd.DataFrame(films_data, columns = ['Films', 'avg_director', 'avg_actors', 'score'])

In [35]:
X

,Films,avg_director,avg_actors,score
0,/film/lego-film-betmen-2017-843480/\n,6.155000,6.900750,6.947
1,/film/obitel-zla-poslednyaya-glava-2016-726753/\n,6.060000,7.450000,5.439
2,/film/gulyay-vasya-2016-946883/\n,6.540000,6.286750,6.538
3,/film/velikaya-stena-2016-611822/\n,7.466000,6.788333,6.055
4,/film/lunnyy-svet-2016-939981/\n,6.483333,5.530000,6.126
5,/film/954059/\n,7.411111,7.093750,7.231
6,/film/844068/\n,5.770000,5.875104,4.489
7,/film/1008216/\n,4.965000,4.875708,4.567
8,/film/logan-2017-807682/\n,7.417143,6.595833,7.428
9,/film/superalibi-2017-977842/\n,6.417143,6.461667,6.370
